In [1]:
# !pip install aiohttp
# !pip install nest-asyncio

In [6]:
import aiohttp
import asyncio
import pandas as pd

import nest_asyncio
nest_asyncio.apply()

In [7]:
def get_price_history(ticker, data):
    df = pd.read_html(data, header=0, index_col=0, parse_dates=True)[0]["Close"]
    df.dropna(inplace=True)
    df.rename(ticker, inplace=True)
    return df

In [8]:
async def get_response(ticker, session):
    url = f"http://performance.morningstar.com/perform/Performance/stock/price-history.action?&t={ticker}&pd=max"
    async with session.get(url) as response:
        data = await response.text()
        return get_price_history(ticker, data)

In [9]:
async def main(tickers):
    async with aiohttp.ClientSession() as session:
        return pd.concat(
            await asyncio.gather(*[
                                   asyncio.create_task(get_response(ticker, session))
                                   for ticker in tickers
                                   ]),
            axis=1
            ).sort_index(ascending=False)

In [10]:
tickers = [
           "AAPL", "MSFT", "AMZN", "GOOGL", "FB", "INTC", "AMD", "MU", "IBM", "HPQ",
           "COST", "LOW", "TGT", "WMT", "HD", "BBY", "HON", "KO", "PEP", "MCD",
           "XOM", "CVX", "MA", "V", "PYPL", "CAT", "DE", "MMM", "TJX", "WM",
           "UNH", "ABBV", "ABT", "AMGN", "MRK", "PFE", "LLY", "TMO", "BMY", "PG",
           "JNJ", "CL", "NKE"
           ]

asyncio.run(main(tickers)).to_csv("value-portfolio-stock-prices.csv")